In [1]:
# ref: https://www.kaggle.com/jacklinggu/lstm-with-glove-embedding-public-lb-score-0-049

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from keras.models import Model, load_model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout,GlobalAveragePooling1D,Conv1D
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 40000
maxlen = 150

def clean_text( text ):
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    #
    return text

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

list_sentences_train = train["comment_text"].fillna("CVxTz").apply(clean_text).values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").apply(clean_text).values
print(y.shape)

Using TensorFlow backend.


(159571, 6)


In [2]:
list_sentences_train[:5]

array([ 'explanation why the edits made under my username hardcore metallica fan were reverted they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now 89 205 38 27',
       'd aww ! he matches this background colour i am seemingly stuck with thanks talk 21 51 january 11 2016 utc ',
       'hey man i am really not trying to edit war it just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info ',
       ' more i cannot make any real suggestions on improvement - i wondered if the section statistics should be later on or a subsection of types of accidents - i think the references may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no - one else does first - if you have any preferences for formatting styl

In [3]:
print('test len',len(test))

test len 153164


In [4]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

print(X_train.shape,X_test.shape)

(159571, 150) (153164, 150)


In [5]:
# check word_index
tmp_cnt = 0
for k in tokenizer.word_index:
    print(k,tokenizer.word_index[k])
    tmp_cnt += 1
    if tmp_cnt >5:
        break
word_idx = tokenizer.word_index

indications 12616
9058989 120503
272632936 138645
hypno 45999
smsmo 175998
ratite 105916


In [6]:
# read word2vec
# 
word_vec_dict = {}
with open('../glove.840B.300d.txt') as f:
    for line in f:
        v_list = line.strip().split(' ')
        k = str(v_list[0])
        v = np.array([float(x) for x in v_list[1:]])
        word_vec_dict[k] = v
print(len(word_vec_dict))
# print(word_vec_dict['is'])
# print(word_vec_dict['are'])

print('Preparing embedding matrix')
EMBEDDING_DIM = 300
nb_words = min(max_features,len(word_idx))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word,i in word_idx.items():
    if i >= max_features:
        continue
    else:
        if word in word_vec_dict:
            embedding_matrix[i] = word_vec_dict[word]
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
del word_vec_dict

2196007
Preparing embedding matrix
Null word embeddings: 3388


In [7]:
from sklearn.metrics import log_loss,accuracy_score
from keras.layers import Bidirectional, Dropout, CuDNNGRU

def eval_val(y,train_x):
    res = 0
    acc_res = 0
    for i in range(6):
        curr_loss = log_loss(y[:,i],train_x[:,i])
        acc = accuracy_score(y[:,i],train_x[:,i].round())
        print(i,curr_loss,acc)
        res += curr_loss
        acc_res += acc
    print('final',res/6, acc_res/6)

# https://github.com/PavelOstyakov/toxic/blob/master/toxic/model.py
def get_model(comp=True):
    inp = Input(shape=(maxlen, ))
    x = Embedding(nb_words, EMBEDDING_DIM, weights=[embedding_matrix],trainable=False)(inp)
    x = Bidirectional(CuDNNGRU(128, return_sequences=True))(x)
    x = Dropout(0.1)(x)
    x = Bidirectional(CuDNNGRU(128, return_sequences=False))(x)
    x = Dense(32, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    if comp:
        model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

    return model


print('def model done')

def model done


In [8]:
tmp_m = get_model(False)
tmp_m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 300)          12000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 150, 256)          330240    
_________________________________________________________________
dropout_1 (Dropout)          (None, 150, 256)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               296448    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                8224      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
__________

In [11]:
from sklearn.utils import shuffle

def del_data_ratio(x,y,ratio=0.8):
    print(x.shape)
    pos_index = np.where(y[:,0]==1)[0]
    neg_index = np.where(y[:,0]==0)[0]
    print(pos_index)
    data_cnt = len(pos_index)
    add_cnt = int(data_cnt*ratio)
    add_index = pos_index[:add_cnt]
    add_x = np.concatenate([x[add_index],x[neg_index]])
    add_y = np.concatenate([y[add_index],y[neg_index]])
    print(add_x.shape,data_cnt)
    add_x,add_y = shuffle(add_x,add_y,random_state=666)
    return add_x,add_y

from sklearn.model_selection import KFold
def kf_train(fold_cnt=3,rnd=1):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((159571,6)),np.zeros((153164,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        
        # change pos ratio
        new_curr_x,new_curr_y = del_data_ratio(curr_x,curr_y)
        new_hold_x,new_hold_y = del_data_ratio(hold_out_x,hold_out_y)
        
        # model
        model = get_model()
        batch_size = 256
        epochs = 8
        file_path="weights_base.best.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit(new_curr_x, new_curr_y, 
                  batch_size=batch_size, epochs=epochs, 
                  validation_data=(new_hold_x,new_hold_y), 
                  callbacks=callbacks_list)
        
        model = load_model(file_path)
        y_test = model.predict(X_test)
        test_pred += y_test
        hold_out_pred = model.predict(hold_out_x)
        train_pred[test_index] = hold_out_pred
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    print('all eval',eval_val(y,train_pred))
    return train_pred, test_pred


print('def done')

def done


In [12]:
import pickle
sample_submission = pd.read_csv("../input/sample_submission.csv")

for f in [3,4]:
    train_pred,test_pred = kf_train(fold_cnt=f)
    print(train_pred.shape,test_pred.shape)    
    sample_submission[list_classes] = test_pred
    sample_submission.to_csv("../results/cudnn_gru_glove_1_sample_{}.gz".format(f), index=False, compression='gzip')
    with open('../features/cudnn_gru_glove_1_sample_feat_{}.pkl'.format(f),'wb') as fout:
        pickle.dump([train_pred,test_pred],fout)
    sample_submission.head()


(106380, 150)
[     0      6     11 ..., 106350 106355 106363]
(104358, 150) 10110
(53191, 150)
[    6    12    16 ..., 53167 53181 53182]
(52154, 150) 5184
Train on 104358 samples, validate on 52154 samples
Epoch 1/8
104358/104358 [==============================] - 93s 894us/step - loss: 0.0638 - acc: 0.9788 - val_loss: 0.0407 - val_acc: 0.9849
Epoch 2/8
104358/104358 [==============================] - 93s 894us/step - loss: 0.0402 - acc: 0.9852 - val_loss: 0.0384 - val_acc: 0.9852
Epoch 3/8
104358/104358 [==============================] - 94s 901us/step - loss: 0.0374 - acc: 0.9858 - val_loss: 0.0365 - val_acc: 0.9858
Epoch 4/8
104358/104358 [==============================] - 94s 905us/step - loss: 0.0349 - acc: 0.9866 - val_loss: 0.0384 - val_acc: 0.9858
Epoch 5/8
104358/104358 [==============================] - 95s 910us/step - loss: 0.0329 - acc: 0.9873 - val_loss: 0.0357 - val_acc: 0.9862
Epoch 6/8
104358/104358 [==============================] - 95s 911us/step - loss: 0.0307 - a

117406/117406 [==============================] - 100s 851us/step - loss: 0.0645 - acc: 0.9798 - val_loss: 0.0416 - val_acc: 0.9845
Epoch 2/8
117406/117406 [==============================] - 99s 845us/step - loss: 0.0402 - acc: 0.9853 - val_loss: 0.0373 - val_acc: 0.9856
Epoch 3/8
117406/117406 [==============================] - 99s 846us/step - loss: 0.0368 - acc: 0.9863 - val_loss: 0.0364 - val_acc: 0.9858
Epoch 4/8
117406/117406 [==============================] - 99s 846us/step - loss: 0.0345 - acc: 0.9869 - val_loss: 0.0363 - val_acc: 0.9860
Epoch 5/8
117406/117406 [==============================] - 100s 851us/step - loss: 0.0323 - acc: 0.9876 - val_loss: 0.0370 - val_acc: 0.9859
Epoch 6/8
117406/117406 [==============================] - 100s 848us/step - loss: 0.0302 - acc: 0.9882 - val_loss: 0.0363 - val_acc: 0.9860
Epoch 7/8
117406/117406 [==============================] - 100s 849us/step - loss: 0.0274 - acc: 0.9892 - val_loss: 0.0375 - val_acc: 0.9857
Epoch 8/8
117406/117406 [=

Epoch 3/8
117385/117385 [==============================] - 100s 851us/step - loss: 0.0354 - acc: 0.9864 - val_loss: 0.0366 - val_acc: 0.9862
Epoch 4/8
117385/117385 [==============================] - 99s 845us/step - loss: 0.0332 - acc: 0.9871 - val_loss: 0.0368 - val_acc: 0.9864
Epoch 5/8
117385/117385 [==============================] - 99s 845us/step - loss: 0.0308 - acc: 0.9879 - val_loss: 0.0368 - val_acc: 0.9865
Epoch 6/8
117385/117385 [==============================] - 99s 846us/step - loss: 0.0287 - acc: 0.9886 - val_loss: 0.0385 - val_acc: 0.9863
Epoch 7/8
117385/117385 [==============================] - 99s 846us/step - loss: 0.0263 - acc: 0.9895 - val_loss: 0.0390 - val_acc: 0.9860
Epoch 8/8
117385/117385 [==============================] - 99s 847us/step - loss: 0.0237 - acc: 0.9906 - val_loss: 0.0417 - val_acc: 0.9856
-------------------------------
0 0.0912206057271 0.965914859216
1 0.0221931961327 0.990549661279
2 0.0468433478495 0.981914006931
3 0.00911978713362 0.9971548

125203/125203 [==============================] - 103s 821us/step - loss: 0.0342 - acc: 0.9867 - val_loss: 0.0355 - val_acc: 0.9864
Epoch 5/8
125203/125203 [==============================] - 103s 820us/step - loss: 0.0320 - acc: 0.9873 - val_loss: 0.0366 - val_acc: 0.9864
Epoch 6/8
125203/125203 [==============================] - 102s 819us/step - loss: 0.0297 - acc: 0.9881 - val_loss: 0.0364 - val_acc: 0.9863
Epoch 7/8
125203/125203 [==============================] - 103s 821us/step - loss: 0.0275 - acc: 0.9888 - val_loss: 0.0394 - val_acc: 0.9866
Epoch 8/8
125203/125203 [==============================] - 103s 821us/step - loss: 0.0248 - acc: 0.9899 - val_loss: 0.0387 - val_acc: 0.9864
(127657, 150)
[     6     12     16 ..., 127627 127632 127640]
(125206, 150) 12252
(31914, 150)
[   10    23    58 ..., 31886 31906 31911]
(31305, 150) 3042
Train on 125206 samples, validate on 31305 samples
Epoch 1/8
125206/125206 [==============================] - 106s 847us/step - loss: 0.0647 - acc: 

ResourceExhaustedError: OOM when allocating tensor with shape[150,256,128]
	 [[Node: training_23/Adam/gradients/bidirectional_28/strided_slice_1_grad/StridedSliceGrad = StridedSliceGrad[Index=DT_INT32, T=DT_FLOAT, _class=["loc:@bidirectional_28/strided_slice_1"], begin_mask=0, ellipsis_mask=0, end_mask=0, new_axis_mask=0, shrink_axis_mask=1, _device="/job:localhost/replica:0/task:0/gpu:0"](training_23/Adam/gradients/bidirectional_28/strided_slice_1_grad/Shape, bidirectional_28/strided_slice_1/stack, bidirectional_28/strided_slice_1/stack_1, bidirectional_28/strided_slice_1/stack_2, training_23/Adam/gradients/bidirectional_28/concat_2_grad/Slice_1)]]

Caused by op 'training_23/Adam/gradients/bidirectional_28/strided_slice_1_grad/StridedSliceGrad', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages